In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


In [6]:
# Function to extract Product Title
def get_title(data):

    try:
        
        title = data.find("span", attrs = {"id":'productTitle'}).text.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(data):

    try:
        price = data.find("span", attrs = {"class":'a-price-whole'})
    
    except:
        price = ""

    return price

# Function to extract Product Rating
def get_rating(data):

    try:
        rating = data.find("span", attrs = {"class":'a-icon-alt'}).text
        
    except:
        rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(data):
    try:
        review_count = data.find("span", attrs = {"id":'acrCustomerReviewText'}).text
        

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(data):
    try:
        available = data.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available


In [ ]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0', 'Accept-Language': 'en-US, en;q=0.5'}) #add your user agent 


    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&crid=3W4LSZYE3M9TA&sprefix=playstation+4%2Caps%2C376&ref=nb_sb_ss_ts-doa-p_1_13"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    data = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = data.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(data))
        d['price'].append(get_price(data))
        d['rating'].append(get_rating(data))
        d['reviews'].append(get_review_count(data))
        d['availability'].append(get_availability(data))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [ ]:
amazon_df